In [1]:
from utilities.landxmlSDK.landxml import landxml
from utilities.landxmlSDK.dcmgeometry.geometry import Geometries
from utilities.landxmlSDK.dcmgeometry.polygons import PolygonGeom
from utilities.landxmlSDK.dcmgeometry.points import PointGeom
from utilities.landxmlSDK.geometryfunctions.transformationfunctions import helmert_trans_unknown
from utilities.landxmlSDK.geometryfunctions.bearingdistancefunctions import calc_distance, angle_differences, \
                                                                            process_angles
from utilities.landxmlSDK.geometryfunctions.otherfunctions import previous_and_next
from pathlib import Path
import geopandas as gpd
import shapely.geometry as sg
from shapely.validation import make_valid
import math
import numpy as np

In [64]:
def remove_stroked_curves(geom):
    """removes stroked curves from a polygon or linestring, this function eliminates nodes from the original geometry
    and returns an altered geometry that contains chords. This could be cleaned up a bit, should also convert
    lat longs to coords and back again to help with the distance and simplification checks"""

    poly = False
    fgs = []
    if not isinstance(geom, (sg.MultiPolygon, sg.MultiLineString)):
        geoms = [geom]
        multi = False
    else:
        geoms = geom.geoms
        multi = True
    for geom in geoms:
        to_remove = set()
        if isinstance(geom, sg.Polygon):
            poly = True
            geom = sg.LineString(geom.exterior.coords)

        if len(geom.coords[:]) > 7:
            geom = geom.simplify(.01, preserve_topology=True)

            points = {}
            coords = {}
            count = 0
            for point in geom.coords[:]:
                points[count] = sg.Point(point)
                coords[count] = point
                count += 1

            pnt_angles = []
            for frm, pnt, nxt in previous_and_next(sorted(points.keys())):
                d = process_angles(frm, pnt, nxt, sorted(points.keys()), points)
                d['point'] = coords.get(pnt)
                pnt_angles.append(d)

            end_curve = set()
            active = False
            for pnt in pnt_angles:                
                if pnt['point'] not in to_remove:
                    pc = pnt['est_centre']
                    pr = pnt['est_radius']
                    for nxt in pnt_angles[pnt_angles.index(pnt)+1:]:
                        nc = nxt['est_centre']
                        nr = nxt['est_radius']
                        nxt_centre_distance = calc_distance(pc, nc)                    
                        nxt_rd_dif = abs(pr - nr)
                        if nxt_centre_distance < 20 and nxt_rd_dif < 20 and abs(180-nxt['angle']) < 20:
                            if abs(180-pnt['angle']) < 20 and pnt['point'] not in end_curve:
                                to_remove.add(pnt['point'])
                            to_remove.add(nxt['point'])
                            active = True
                        else:
                            if active is True:
                                end_curve.add(nxt['point'])
                            active = False
                            break

        
        if poly is True:
            ppoints = [p for p in geom.coords[:] if p not in to_remove]
            ppoints += [ppoints[0]]
            new_geom = sg.Polygon(ppoints)
        else:
            new_geom = sg.LineString([p for p in geom.coords[:] if p not in to_remove])

        if new_geom.is_valid is True:
            fgs.append(new_geom)
        else:
            pp = make_valid(new_geom)
            # handle point (perfect circle) or 2 line polygons after stroked curve is removed
            if isinstance(pp, (sg.LineString, sg.Point)) and poly is True:
                pp = sg.Polygon(geom.coords[:] + [geom.coords[:][0]])
            fgs.append(pp)

    if multi is True:
        if poly is True:
            fgs = sg.MultiPolygon(fgs)
        else:
            fgs = sg.MultiLineString(fgs)
    else:
        # return just the first geometry if it wasnt multi to begin with
        fgs = fgs[0]

    return fgs


In [65]:
polygons = gpd.read_file('/Users/jamesleversha/Downloads/test_area_queenscliffe/test_area/queenscliffe_all_parcels.shp')
polygons = polygons.to_crs(7855)

In [66]:
subset = polygons#.loc[polygons['SPI'] == 'CM\PS326476'].copy()

In [67]:
subset.geometry = subset.geometry.apply(remove_stroked_curves)


In [68]:
subset.to_file('/Users/jamesleversha/Downloads/test.gpkg', driver='GPKG')

IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException:

IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException:

IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException:

IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException:

IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException:

...

IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException:

IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException:

IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException:

IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException:

IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: Geometry must be a Point or LineString
IllegalArgumentException: